<a href="https://colab.research.google.com/github/MAGalarza/Encriptado-Challenger/blob/master/Cumulos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

resumamos lo que hace el código para procesar los datos de "Cumulos":

Carga y preparación de datos:

Carga datos de un archivo Excel ("CUMULO_CAUCION-20250131_Ori.xlsx") en un Pandas DataFrame datosllamado
Crea una copia del DataFrame nombrado nuevos_datospara las modificaciones.
Converts specific columns (CUIT_TOMADOR, NOMBRE_TOMADOR, APELLIDO_TOMADOR, ID_PRODUCTOR, DE_PRODUCTOR) to string type.
Creando nuevas columnas:

Creates a new column 'NOMBRE_COMPLETO_TOMADOR' by concatenating CUIT_TOMADOR, NOMBRE_TOMADOR, and APELLIDO_TOMADOR.
Crea 'CONTROL_TC' dividiendo 'IM_SUMA_INSURE_MF' por 'IM_SUMA_INSURE_MO'.
Creates 'PAS_COMPLETO' by concatenating 'ID_PRODUCTOR' and 'DE_PRODUCTOR'.
Crea 'ISSUE_TYPE' basado en los valores en 'DESC_ENDOSO' Si 'DESC_ENDOSO' es nulo, asigna 'Valor predeterminado'; De lo contrario, utiliza el valor de 'DESC_ENDOSO'
Extrae el mes ('MES') y el año ('AÑO') de 'FECHAEMI' y crea nuevas columnas para ellos.
Agrega una nueva columna 'PERIODO' con un valor fijo (por ejemplo, '2025-01').
Cálculos y transformaciones:

Calcula 'PRIMA_PESIFICADA' en función de 'PRIMA_MO' y 'CONTROL_TC'. Si 'PRIMA_MO' es 0, asigna 0; en caso contrario, multiplica 'PRIMA_MO' por 'CONTROL_TC'.
Calcula 'DERECHO_EMISION' restando 'GASTO_NOTARIAL' de 'DERECHO_EMISION_CAL_MO' y luego multiplicando el resultado por 'CONTROL_TC'.
Creates 'CODIGO_RIESGO' by concatenating 'ID_PRODUCTO', 'ID_COBERTURA', and 'ID_RIESGO'.
Filtrado y agregación:

Filtra el DataFrame para seleccionar datos de un mes y año específicos usando 'FECHAEMI'.
Agrupa los datos filtrados por 'RIESGO' y calcula:
The sum of 'DERECHO_EMISION' (named 'Suma_Derecho_Emision').
The count of 'POLIZA' (named 'Cantidad_Poliza' or 'Cantidad_Polizas').
Formatos 'Issue_Rights_Sum' para mostrar en miles (por ejemplo, '1,234K') y 'Policy_Amount' para mostrar sin decimales
Producción:

Prints the results, which include the 'RIESGO', 'Suma_Derecho_Emision', and 'Cantidad_Polizas/Cantidad_Poliza'.
En esencia, el código procesa datos de "Cumulos" para crear nuevas variables, realizar cálculos, filtrar por fecha, agregar por riesgo y presentar resultados resumidos para el período deseado.

In [ ]:
import pandas as pd
import numpy as np
datos = pd.read_excel("CUMULO_CAUCION-20250131_Ori.xlsx")
datos.head()

In [ ]:
datos.info()

In [ ]:
datos.describe()

In [ ]:
# ... (código para cargar tus datos en un DataFrame llamado 'df') ...
nuevos_datos = datos.copy()

# 1. Convertir las columnas a tipo string (texto)
nuevos_datos['CUIT_TOMADOR'] = nuevos_datos['CUIT_TOMADOR'].astype(str)
nuevos_datos['NOMBRE_TOMADOR'] = nuevos_datos['NOMBRE_TOMADOR'].astype(str)
nuevos_datos['APELLIDO_TOMADOR'] = nuevos_datos['APELLIDO_TOMADOR'].astype(str)

# 2. Realizar la concatenación
nuevos_datos['NOMBRE_COMPLETO_TOMADOR'] = (nuevos_datos['CUIT_TOMADOR'].str.cat(nuevos_datos['NOMBRE_TOMADOR'], sep=' - ').str.cat(nuevos_datos['APELLIDO_TOMADOR'], sep=' '))

# 3. Mostrar las primeras filas para verificar
print(nuevos_datos[['CUIT_TOMADOR', 'NOMBRE_TOMADOR', 'APELLIDO_TOMADOR', 'NOMBRE_COMPLETO_TOMADOR']].head())

In [ ]:
# Hace el control del TC y actualiza el nuevo archivo

nuevos_datos['CONTROL_TC'] = nuevos_datos['IM_SUMA_ASEGURADA_MF'] / nuevos_datos['IM_SUMA_ASEGURADA_MO']
nuevos_datos

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos['PAS_COMPLETO'] = nuevos_datos['ID_PRODUCTOR'].astype(str).str.cat(nuevos_datos['DE_PRODUCTOR'].astype(str), sep=' - ')

# 1. Convertir las columnas a tipo string (texto)
# nuevos_datos['ID_PRODUCTOR'] = nuevos_datos['ID_PRODUCTOR'].astype(str)
# nuevos_datos['DE_PRODUCTOR'] = nuevos_datos['DE_PRODUCTOR'].astype(str)


# 2. Realizar la concatenación
# nuevos_datos['PAS_COMPLETO'] = (nuevos_datos['ID_PRODUCTOR'].str.cat(nuevos_datos['DE_PRODUCTOR'], sep=' - '))

# 3. Mostrar las primeras filas para verificar
print(nuevos_datos[['ID_PRODUCTOR', 'DE_PRODUCTOR', 'PAS_COMPLETO']].head())

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos.head()

In [ ]:
nuevos_datos = nuevos_datos.assign(TIPO_EMISION=None)
nuevos_datos.loc[nuevos_datos['DESC_ENDOSO'].isnull(), 'TIPO_EMISION'] = 'Valor por defecto'
nuevos_datos.loc[nuevos_datos['DESC_ENDOSO'].notnull(), 'TIPO_EMISION'] = nuevos_datos['DESC_ENDOSO']

# Para verificar los cambios:
print(nuevos_datos[['DESC_ENDOSO', 'TIPO_EMISION']].head(10))

In [ ]:
nuevos_datos.head()

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos['MES'] = pd.to_datetime(nuevos_datos['FECHAEMI']).dt.month

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos.head()

In [ ]:
nuevos_datos['AÑO'] = pd.to_datetime(nuevos_datos['FECHAEMI']).dt.year

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos.head()

In [ ]:
nuevos_datos['PERIODO'] = '2025-01'  # Reemplaza '202312' con tu período que se está trabajando

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos.head()

In [ ]:
import numpy as np

nuevos_datos['PRIMA_PESIFICADA'] = np.where(nuevos_datos['PRIMA_MO'] == 0, 0, nuevos_datos['PRIMA_MO'] * nuevos_datos['CONTROL_TC'])

In [ ]:
print(nuevos_datos[['PRIMA_MO', 'CONTROL_TC', 'PRIMA_PESIFICADA']].head())

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos['DERECHO_EMISION'] = (nuevos_datos['DERECHO_EMISION_CAL_MO'] - nuevos_datos['GASTO_NOTARIAL']) * nuevos_datos['CONTROL_TC']

In [ ]:
print(nuevos_datos[['DERECHO_EMISION_CAL_MO', 'GASTO_NOTARIAL', 'CONTROL_TC', 'DERECHO_EMISION']].head())

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos['CODIGO_RIESGO'] = nuevos_datos['ID_PRODUCTO'].astype(str) + '-' + nuevos_datos['ID_COBERTURA'].astype(str) + '-' + nuevos_datos['ID_RIESGO'].astype(str)

In [ ]:
print(nuevos_datos[['ID_PRODUCTO', 'ID_COBERTURA', 'ID_RIESGO', 'CODIGO_RIESGO']].head())

In [ ]:
nuevos_datos.info()

In [ ]:
suma_derecho_emision_por_riesgo = nuevos_datos.groupby('RIESGO')['DERECHO_EMISION'].sum()

print(suma_derecho_emision_por_riesgo)

In [ ]:
# Assuming 'FECHAEMI' column contains the date
mes_deseado = 1  # Replace with the desired month (1 for January, 2 for February, etc.)
año_deseado = 2025  # Replace with the desired year

# Filter the DataFrame
datos_filtrados = nuevos_datos[
    (pd.to_datetime(nuevos_datos['FECHAEMI']).dt.month == mes_deseado) &
    (pd.to_datetime(nuevos_datos['FECHAEMI']).dt.year == año_deseado)
]

# Calculate the sum
suma_derecho_emision_por_riesgo = datos_filtrados.groupby('RIESGO')['DERECHO_EMISION'].sum()

print(suma_derecho_emision_por_riesgo)

In [ ]:
# ... (previous code for filtering by month and year) ...

# Calculate the sum of 'Derecho de emision' and count of 'POLIZA'
resultados = datos_filtrados.groupby('RIESGO').agg(
    Suma_Derecho_Emision=('DERECHO_EMISION', 'sum'),
    Cantidad_Poliza=('POLIZA', 'count')  # Added this line
)

print(resultados)

In [ ]:
# ... (previous code for filtering, grouping, and aggregation) ...

# Format 'Suma_Derecho_Emision' in thousands
resultados['Suma_Derecho_Emision'] = resultados['Suma_Derecho_Emision'].map(lambda x: '{:,.0f}K'.format(x/1000))

print(resultados)

In [ ]:
# ... (previous code for filtering by month and year) ...

# Calculate the sum of 'Derecho de emision' and count of 'POLIZA'
resultados = datos_filtrados.groupby('RIESGO').agg(
    Cantidad_Poliza=('POLIZA', 'count'),
    Suma_Derecho_Emision=('DERECHO_EMISION', lambda x: (x * 1000).sum())  # Apply the multiplication within the aggregation function

)

print(resultados)

In [ ]:
# ... (previous code for filtering by month and year) ...

# Calculate the sum of 'Derecho de emision' and count of 'POLIZA'
resultados = datos_filtrados.groupby('RIESGO').agg(
    Cantidad_Poliza=('POLIZA', 'count'),
    Suma_Derecho_Emision=('DERECHO_EMISION', 'sum')  # Calculate the sum first
)

# Format 'Suma_Derecho_Emision' in thousands *after* aggregation
resultados['Suma_Derecho_Emision'] = resultados['Suma_Derecho_Emision'].map(lambda x: '{:,.0f}K'.format(x/1000))

print(resultados)

In [ ]:


# Calculate the sum of 'Derecho de emision' and count of 'POLIZA'
resultados = datos_filtrados.groupby('RIESGO').agg(
    Cantidad_Polizas=('POLIZA', 'count'),
    Suma_Derecho_Emision=('DERECHO_EMISION', 'sum')  # Calculate the sum first
)

# Format 'Suma_Derecho_Emision' in thousands *after* aggregation
resultados['Suma_Derecho_Emision'] = resultados['Suma_Derecho_Emision'].map(lambda x: '{:,.0f}'.format(x/1))

# Add total row
resultados.loc['Total'] = resultados.sum(numeric_only=True)  # Calculate sum for numeric columns
resultados.loc['Total', 'Suma_Derecho_Emision'] = '{:,.0f}'.format(resultados.loc['Total', 'Suma_Derecho_Emision'] /1)
#resultados.at['Total', 'Suma_Derecho_Emision'] = resultados['Suma_Derecho_Emision'].sum()


print(resultados)

Explicación de los cambios:

Calcular el total primero: el total de 'Suma_Derecho_Emision' se calcula antes de aplicar cualquier formato. Esto garantiza que trabajemos con valores numéricos para el cálculo del total.
Formato después del cálculo del total: el formato que se utiliza '{:,.0f}'.format(x/1)se aplica después de calcular el total y agregarlo a la fila "Total". De esta manera, formateamos los valores numéricos finales, incluido el total.
Al calcular el total antes de formatear y luego aplicar el formato a toda la columna, incluido el total, evitamos problemas con la suma de cadenas formateadas. Esto debería brindarle el 'Total' correcto para 'Suma_Derecho_Emision'.

In [ ]:
# ... (previous code for filtering by month and year) ...

# Calculate the sum of 'Derecho de emision' and count of 'POLIZA'
resultados = datos_filtrados.groupby('RIESGO').agg(
    Cantidad_Polizas=('POLIZA', 'count'),
    Suma_Derecho_Emision=('DERECHO_EMISION', 'sum')  # Calculate the sum first
)

# Format 'Suma_Derecho_Emision' in thousands *after* aggregation and calculating total
resultados.loc['Total'] = resultados.sum(numeric_only=True)  # Calculate sum for numeric columns

# Now apply formatting to the 'Suma_Derecho_Emision' column including Total row
resultados['Suma_Derecho_Emision'] = resultados['Suma_Derecho_Emision'].map(lambda x: '{:,.0f}'.format(x/1))

print(resultados)

In [ ]:
# ... (previous code for filtering by month and year) ...

# Calculate the sum of 'Derecho de emision' and count of 'POLIZA'
resultados = datos_filtrados.groupby('RIESGO').agg(
    Cantidad_Polizas=('POLIZA', 'count'),
    Suma_Derecho_Emision=('DERECHO_EMISION', 'sum')  # Calculate the sum first
)

# Format 'Suma_Derecho_Emision' in thousands *after* aggregation and calculating total
resultados.loc['Total'] = resultados.sum(numeric_only=True)  # Calculate sum for numeric columns

# Now apply formatting to both columns including Total row
resultados['Suma_Derecho_Emision'] = resultados['Suma_Derecho_Emision'].map(lambda x: '{:,.0f}'.format(x/1))
resultados['Cantidad_Polizas'] = resultados['Cantidad_Polizas'].map(lambda x: '{:,.0f}'.format(x))  # Format without cents

print(resultados)

Explicación de los cambios

Formato 'Cantidad_Polizas': Luego de calcular el total y formatear 'Suma_Derecho_Emision', agregamos una línea para formatear 'Cantidad_Polizas' usando resultados['Cantidad_Polizas'] = resultados['Cantidad_Polizas'].map(lambda x: '{:,.0f}'.format(x)).
Esta línea se utiliza .mappara aplicar la cadena de formato {:,.0f}a cada valor en la columna 'Cantidad_Polizas'. - {:,.0f}: Esto especifica el formato: - :,agrega una coma como separador de miles. - .0findica que no se deben mostrar decimales.

Ahora, las columnas 'Suma_Derecho_Emision' y 'Cantidad_Polizas' tendrán el formato deseado en la salida final.